In [3]:
# check correct modules and virtual environment
!printenv | grep VIRTUAL_ENV
!python --version

VIRTUAL_ENV=/cluster/home/jpostels/nnaimi/ingp-env
Python 3.9.9


In [4]:
import sys
import os
import time
import numpy as np
import math

pyngp_path = '/cluster/home/jpostels/nnaimi/instant-ngp/build'
sys.path.append(pyngp_path)
sys.path.append(os.path.abspath("/cluster/home/jpostels/nnaimi/instant-ngp/scripts"))
import pyngp as ngp
from scenes import *

In [5]:
def scaling_parameter(N_max, N_min):
    return math.exp((math.log(N_max)-math.log(N_min)) / (L-1)) 

def encoding_parameter_calculator(L, F, T, N_min, N_max=None, b=None):
    sum = 0
    if b is None and N_max is not None:
        b = scaling_parameter(N_max, N_min)
    elif b is None and N_max is None:
        assert False, "One of either b or N_max has not been provided"
        
    print("scaling param: ", b)
    print("# of levels: ", L)
    
    for l in range(L):
        grid_res = math.floor(N_min*(b**l))
        grid_size = grid_res**2
        sum += min(grid_size, T) * F
    return sum

def get_list_of_grid_resolutions(N_min, b, L):
    return [int(np.floor(N_min*(b ** l))) for l in range(L)]

# Exploratoration on ingp parameters and there effect on PSNR

### What do we want to do?

Goal is to understand how the psnr is affected by the choice of parameters and the allocation thereof

### How do we do this?

We will design the following experiments:
1) Varying the hyperparameters in such a way to keep the total number of parameters constant and analysing the effect of the allocation of parameters on the PSNR
2) Scaling down the total number of parameters to understand the trade-off between reducing the number of parameters for different hyperparameter allocations

#### Experiment ONE:
1) Determine the total amount of degrees of freedom in the image of choice for the study
2) Let the total number of parameters (Encoding + MLP) be the number specified by the degrees of freedom for the image of choice that we are performing our studies on (CxHxW)
3) Assign different hyperparameter allocations to understand the tradeoff between the hyperparameters
4) Scatter plot for psnr and various parameter choice

#### Experiment TWO:
1) Determine the total amount of degrees of freedom in the image of choice for the study
2) Scale down the total number of trainable parameters by factor 2 each time.
3) For each scaled down number of total trainable parameters, assign different hyperparameter variations that add up to the same number of parameters.
4) scatter plots

### Total Number of DoF for KODAK image 11

$$512\times768\times3 = 1'179'648 \text{Bytes}$$ 

We keep the collision resolving MLP ($m(y,\phi)$) fixed with $n_{hidden}=2$, $n_{neurons}=64$ and vary only the hyper parameters of the neural encoding; $T, L, N_{min}, F, b$. We define the parameters as
$$ T := \text{hash-table size} \newline L := \text{# Resolution Levels} \newline N_{min} := \text{Coarsest Resolution} \newline F := \text{feature vector size} \newline b := \text{scaling parameter} = \exp ( \frac{\ln N_{max}- \ln N_{min}}{L-1}) $$

Note: We can either choose to select the min and max resolution, $N_{min}$ and $N_{max}$, or we can choose to select $N_{min}$ and $b$

The total number of encoding parameters is given by 
$$\text{# encoding params} = F \times \sum_{l=0}^{L-1} \min((N_{min}b^{l})^2, T)$$

Example: $L=2, F=2, T=2^18, N_{min}=64, b=2$
$$\text{# encoding params} = 2 \sum_{l=0}^{L-1} \min((N_{min}b^{l})^2, T) = 2 \sum_{l=0}^{1} \min((64 \times 2^{l})^2, 2^18) = 2 \times (64^2 + 128^2) = 40'960$$ 



### Parameter Choices for experiment ONE and Result

| TOTAL | $$L$$ | $$F$$ | $$T$$ | $$N_{min}$$ | $$N_{max}$$ | $$b$$ | PSNR |
|---|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
| 1′048′576 | 1 | 2 | $2^{19}$ | $2^{18}$ | $2^{18}$ | 2 | 17.966 |
| 1′048′576 | 2 | 2 | $2^{18}$ | $2^{17}$ | $2^{18}$ | 2 | 18.202 |
| 1′048′576 | 4 | 2 | $2^{17}$ | $2^{15}$ | $2^{18}$ | 2 | 24.335 |
| 1′048′576 | 8 | 2 | $2^{16}$ | $2^{11}$ | $2^{18}$ | 2 | 37.015 |
| 1′048′576 | 16 | 2 | $2^{15}$ | $2^{8}$ | $2^{23}$ | 2 | 40.996 |
| 1′048′576 | 32 | 2 | $2^{14}$ | $2^{7}$ | $2^{38}$ | 2 | 8.138 |
| 1′048′576 | 64 | 2 | $2^{13}$ | $2^{7}$ | $2^{70}$ | 2 | 8.139 |

### Parameter Choices for experiment TWO and Result

#### n_total: 524'288
| TOTAL | $$L$$ | $$F$$ | $$T$$ | $$N_{min}$$ | $$N_{max}$$ | $$b$$ | PSNR |
|---|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
| 524'288 | 1 | 2 | $2^{18}$ | $2^{9}$ | $2^{9}$ | 2 | 39.886 |
| 524'288 | 2 | 2 | $2^{17}$ | $2^{9}$ | $2^{10}$ | 2 | 33.317 |
| 524'288 | 4 | 2 | $2^{16}$ | $2^{8}$ | $2^{11}$ | 2 | 45.996 |
| 524'288 | 8 | 2 | $2^{15}$ | $2^{8}$ | $2^{15}$ | 2 | 39.967 |
| 524'288 | 16 | 2 | $2^{14}$ | $2^{7}$ | $2^{22}$ | 2 | 39.561 |
| 524'288 | 32 | 2 | $2^{13}$ | $2^{7}$ | $2^{38}$ | 2 | 8.138 |

#### n_total: 262'144
| TOTAL | $$L$$ | $$F$$ | $$T$$ | $$N_{min}$$ | $$N_{max}$$ | $$b$$ | PSNR |
|---|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
| 262'144 | 1 | 2 | $2^{17}$ | $2^{9}$ | $2^{9}$ | 2 | 24.496 |
| 262'144 | 2 | 2 | $2^{16}$ | $2^{8}$ | $2^{9}$ | 2 | 35.351 |
| 262'144 | 4 | 2 | $2^{15}$ | $2^{8}$ | $2^{11}$ | 2 | 28.755 |
| 262'144 | 8 | 2 | $2^{14}$ | $2^{7}$ | $2^{14}$ | 2 | 37.492 |
| 262'144 | 16 | 2 | $2^{13}$ | $2^{7}$ | $2^{22}$ | 2 | 31.689 |

#### n_total: 131'072
| TOTAL | $$L$$ | $$F$$ | $$T$$ | $$N_{min}$$ | $$N_{max}$$ | $$b$$ | PSNR |
|---|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
| 131'072 | 1 | 2 | $2^{16}$ | $2^{8}$ | $2^{8}$ | 2 | 30.819 |
| 131'072 | 2 | 2 | $2^{15}$ | $2^{8}$ | $2^{9}$ | 2 | 24.092 |
| 131'072 | 4 | 2 | $2^{14}$ | $2^{7}$ | $2^{10}$ | 2 | 33.487 |
| 131'072 | 8 | 2 | $2^{13}$ | $2^{7}$ | $2^{14}$ | 2 | 30.454 |
| 131'072 | 16 | 2 | $2^{12}$ | $2^{6}$ | $2^{21}$ | 2 | 34.43 |

#### n_total: 65'536
| TOTAL | $$L$$ | $$F$$ | $$T$$ | $$N_{min}$$ | $$N_{max}$$ | $$b$$ | PSNR |
|---|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
| 65'536 | 1 | 2 | $2^{15}$ | $2^{8}$ | $2^{8}$ | 2 | 22.369 |
| 65'536 | 2 | 2 | $2^{14}$ | $2^{7}$ | $2^{8}$ | 2 | 28.965 |
| 65'536 | 4 | 2 | $2^{13}$ | $2^{7}$ | $2^{10}$ | 2 | 24.616 |
| 65'536 | 8 | 2 | $2^{12}$ | $2^{6}$ | $2^{13}$ | 2 | 29.867 |
| 65'536 | 16 | 2 | $2^{11}$ | $2^{6}$ | $2^{21}$ | 2 | 27.348 |

#### n_total: 32'768
| TOTAL | $$L$$ | $$F$$ | $$T$$ | $$N_{min}$$ | $$N_{max}$$ | $$b$$ | PSNR |
|---|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
| 32'768 | 1 | 2 | $2^{14}$ | $2^{7}$ | $2^{7}$ | 2 | 26.457 |
| 32'768 | 2 | 2 | $2^{13}$ | $2^{7}$ | $2^{8}$ | 2 | 22.469 |
| 32'768 | 4 | 2 | $2^{12}$ | $2^{6}$ | $2^{9}$ | 2 | 28.341 |
| 32'768 | 8 | 2 | $2^{11}$ | $2^{6}$ | $2^{13}$ | 2 | 25.527 |
| 32'768 | 16 | 2 | $2^{10}$ | $2^{5}$ | $2^{14}$ | 2 | 26.81 |

#### n_total: 16'384
| TOTAL | $$L$$ | $$F$$ | $$T$$ | $$N_{min}$$ | $$N_{max}$$ | $$b$$ | PSNR |
|---|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
| 16'384 | 1 | 2 | $2^{13}$ | $2^{7}$ | $2^{7}$ | 2 | 20.944 |
| 16'384 | 2 | 2 | $2^{12}$ | $2^{6}$ | $2^{7}$ | 2 | 25.744 |
| 16'384 | 4 | 2 | $2^{11}$ | $2^{6}$ | $2^{9}$ | 2 | 22.933 |
| 16'384 | 8 | 2 | $2^{10}$ | $2^{5}$ | $2^{12}$ | 2 | 26.247 |
| 16'384 | 16 | 2 | $2^{9}$ | $2^{5}$ | $2^{20}$ | 2 | 25.108 |

#### n_total: 8'192
| TOTAL | $$L$$ | $$F$$ | $$T$$ | $$N_{min}$$ | $$N_{max}$$ | $$b$$ | PSNR |
|---|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
| 8'192 | 1 | 2 | $2^{12}$ | $2^{6}$ | $2^{6}$ | 2 | 24.258 |
| 8'192 | 2 | 2 | $2^{11}$ | $2^{6}$ | $2^{7}$ | 2 | 21.483 |
| 8'192 | 4 | 2 | $2^{10}$ | $2^{5}$ | $2^{8}$ | 2 | 25.457 |
| 8'192 | 8 | 2 | $2^{9}$ | $2^{5}$ | $2^{12}$ | 2 | 23.465 |
| 8'192 | 16 | 2 | $2^{8}$ | $2^{4}$ | $2^{19}$ | 2 | 24.756 |

#### n_total: 4'096
| TOTAL | $$L$$ | $$F$$ | $$T$$ | $$N_{min}$$ | $$N_{max}$$ | $$b$$ | PSNR |
|---|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
| 4'096 | 1 | 2 | $2^{11}$ | $2^{6}$ | $2^{6}$ | 2 | 19.818 |
| 4'096 | 2 | 2 | $2^{10}$ | $2^{5}$ | $2^{6}$ | 2 | 23.923 |
| 4'096 | 4 | 2 | $2^{9}$ | $2^{5}$ | $2^{8}$ | 2 | 21.656 |
| 4'096 | 8 | 2 | $2^{8}$ | $2^{4}$ | $2^{11}$ | 2 | 24.337 |
| 4'096 | 16 | 2 | $2^{7}$ | $2^{4}$ | $2^{19}$ | 2 | 23.841 |



In [34]:
L=8
F=2
T=2**8
N_min=2**1
N_max=None
b=1.9
target=2**12

grid_res = get_list_of_grid_resolutions(N_min=N_min, b=b, L=L)

n_enc_params = encoding_parameter_calculator(L=L, F=F, T=T, N_min=N_min, b=b, N_max=N_max)

print("Total Enc Params: ", n_enc_params)

print("Target #  Params: ", target)

print(n_enc_params == target)

scaling param:  1.9
# of levels:  8
Total Enc Params:  2510
Target #  Params:  4096
False


#### scaling the hash table size: n starting --> table with n_params = 4'096

| TOTAL | $$L$$ | $$F$$ | $$T$$ | $$N_{min}$$ | $$N_{max}$$ | $$b$$ | PSNR |
|---|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
| 4'096  | 8 | 2 | $2^{8}$ | $2^{4}$ | $2^{11}$ | 2 | 24.337 |
| 7'680  | 8 | 2 | $2^{9}$ | $2^{4}$ | $2^{11}$ | 2 | 25.293 |
| 14'848 | 8 | 2 | $2^{10}$ | $2^{4}$ | $2^{11}$ | 2 | 26.68 |
| 27'136 | 8 | 2 | $2^{11}$ | $2^{4}$ | $2^{11}$ | 2 | 28.084 |
| 51'712 | 8 | 2 | $2^{12}$ | $2^{4}$ | $2^{11}$ | 2 | 30.513 |
| 92'672 | 8 | 2 | $2^{13}$ | $2^{4}$ | $2^{11}$ | 2 | 33.082 |
| 174'592 | 8 | 2 | $2^{14}$ | $2^{4}$ | $2^{11}$ | 2 | 37.35 |
| 305'664 | 8 | 2 | $2^{15}$ | $2^{4}$ | $2^{11}$ | 2 | 43.196 |
| 567'808 | 8 | 2 | $2^{16}$ | $2^{4}$ | $2^{11}$ | 2 | 49.696 |


#### varying min resolution: n starting --> table with n_params = 4'096

| TOTAL | $$L$$ | $$F$$ | $$T$$ | $$N_{min}$$ | $$N_{max}$$ | $$b$$ | PSNR |
|---|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
| 4'096 | 8 | 2 | $2^{8}$ | $2^{6}$ | $2^{11}$ | 2 | 21.557 |
| 4'096 | 8 | 2 | $2^{8}$ | $2^{5}$ | $2^{11}$ | 2 | 21.916 |
| 4'096 | 8 | 2 | $2^{8}$ | $2^{4}$ | $2^{11}$ | 2 | 24.366 |
| 3'712 | 8 | 2 | $2^{8}$ | $2^{3}$ | $2^{11}$ | 2 | 24.748 |
| 3'232 | 8 | 2 | $2^{8}$ | $2^{2}$ | $2^{11}$ | 2 | 24.781 |
| 2'728 | 8 | 2 | $2^{8}$ | $2^{1}$ | $2^{11}$ | 2 | 24.802 |
| 2'218 | 8 | 2 | $2^{8}$ | $2^{0}$ | $2^{11}$ | 2 | 24.523 |

#### varying scaling parameter: n starting --> table with n_params = 4'096

| TOTAL | $$L$$ | $$F$$ | $$T$$ | $$N_{min}$$ | $$N_{max}$$ | $$b$$ | PSNR |
|---|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
| 2'728 | 8 | 2 | $2^{8}$ | $2^{1}$ | $2^{XX}$ | 2 | 24.802 |
| 2'624 | 8 | 2 | $2^{8}$ | $2^{1}$ | $2*1.9^{XX}$ | 1.9 | 24.825 |
| 2'496 | 8 | 2 | $2^{8}$ | $2^{1}$ | $2*1.8^{XX}$ | 1.8 | 24.77 |
| 2'078 | 8 | 2 | $2^{8}$ | $2^{1}$ | $2*1.6^{XX}$ | 1.6 | 24.609 |
| 1'568 | 8 | 2 | $2^{8}$ | $2^{1}$ | $2*1.4^{XX}$ | 1.4 | 24.129 |


#### varying scaling parameter: n starting --> table with n_params = 4'096

| TOTAL | $$L$$ | $$F$$ | $$T$$ | $$N_{min}$$ | $$N_{max}$$ | $$b$$ | PSNR |
|---|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
| 32'768 | 4 | 2 | $2^{12}$ | $2^{6}$ | $2^{9}$ | 2 | 28.341 |
| 32'768 | 4 | 2 | $2^{12}$ | $2^{6}$ | $2^{9}$ | 1.8 | 28.399 |
| 32'768 | 4 | 2 | $2^{12}$ | $2^{6}$ | $2^{9}$ | 1.6 | 28.614 |
| 32'768 | 4 | 2 | $2^{12}$ | $2^{6}$ | $2^{9}$ | 1.4 | 28.613 |
| 32'768 | 4 | 2 | $2^{12}$ | $2^{6}$ | $2^{9}$ | 1.2 | 28.347 |

In [5]:
# !ls /cluster/work/cvl/jpostels/nnaimi/output

In [6]:
def get_scene(scene):
    for scenes in [scenes_sdf, scenes_nerf, scenes_image, scenes_volume]:
        if scene in scenes:
            return scenes[scene]
    return None

In [7]:
testbed = ngp.Testbed()

23:07:33 SUCCESS  Initialized CUDA. Active GPU is #0: NVIDIA GeForce RTX 3090 [86]


In [8]:
snapshot = '/cluster/work/cvl/jpostels/nnaimi/output/kodim11-20230718-113930.ingp'
print(snapshot)

/cluster/work/cvl/jpostels/nnaimi/output/kodim11-20230718-113930.ingp


In [9]:
scene_info = get_scene(snapshot)
testbed.load_snapshot(snapshot)
time.sleep(0.2)

print(testbed.n_params())
level_params = [testbed.n_encoding_level_params(l) for l in range(2)]

print("Grid Resolutions per Level (before saturation): ", get_list_of_grid_resolutions(64, 4, 2))
print("Entries in hash table per resolution: ", level_params)
print("Total Encoding parameters according to formula: ", sum(level_params)*2)

23:07:33 INFO     Loading network snapshot from: /cluster/work/cvl/jpostels/nnaimi/output/kodim11-20230718-113930.ingp
23:07:33 INFO     GridEncoding:  Nmin=32768 b=2 F=2 T=2^17 L=4
23:07:33 INFO     Model:         2--[HashGrid]-->16--[FullyFusedMLP(neurons=64,layers=4)]-->3
23:07:33 INFO       total_encoding_params=1048576 total_network_params=6144
1054720
Grid Resolutions per Level (before saturation):  [64, 256]
Entries in hash table per resolution:  [131072, 131072]
Total Encoding parameters according to formula:  524288


In [10]:
from sympy import *
x, y, z = symbols("x y z")

Nl = Symbol('N_l')
Nmin = Symbol('N_{min}')
Nmax = Symbol('N_{max}')
b = Symbol('b')
L = Symbol('L')
l = Symbol('l')

In [11]:
level_resolution = floor(Nmin * b ** l)
level_resolution

floor(N_{min}*b**l)